# Dekoratory

Ogólna idea - modyfikacja działania funkcji bez ingerowania w tę funkcję. Przykłady:
- dodanie mierzenia czasu
- dodanie logowania do pliku
- obsługa błędów

Często nie chcemy modyfikować kodu funkcji, chociażby dlatego, żeby nie zmniejszać czytelności. Albo jeśli pewne zmiany chcemy wprowadzić do wielu funkcji to zamiast poprawiać w każdej, lepiej napisać jedną modyfikację.

## Przykład 1 – dzielenie przez 0

In [2]:
def divide(a, b):
    return a / b

In [4]:
divide(6, 0)  # divide(6, 0) błąd

ZeroDivisionError: division by zero

Funkcja spełnia swoje zadanie, ale nie ma obsługi błędów. W takim razie modyfikujemy jej ciało

In [10]:
def divide(a, b):
    if b == 0:
        return None
    else:
        return a / b

In [11]:
divide(6, 0) is None

True

Modyfikacja obsługuje błąd, ale wymaga zmiany w funkcji. Zamiast tego, obsługę błędów można obsłużyć w innej funkcji

In [12]:
def divide(a, b):
    return a / b

Poniższe funkcja przyjmuje jako argument funkcję `divide` i zwraca jej zmodyfikowaną wersję.

In [13]:
def modify_division_function_to_handle_division_by_zero(divide_function): 
    def wrapper(a, b):  # wrapper to zmodyfikowana wersja funkcji
        if b == 0:
            return None
        else:
            return divide_function(a, b)  # nie duplikujemy kodu z funkcji dzielącej
    
    # Skąd w tym miejscu wiadomo czym jest a i b?
    # Nie wiadomo. po prostu zwracamy funkcję, która będzie miała takie parametry
    
    return wrapper  # zwraca ulepszoną funkcję

In [15]:
divide(6, 0)  # divide(6, 0) spowoduje błąd

ZeroDivisionError: division by zero

In [16]:
modify_division_function_to_handle_division_by_zero(divide)  # to zwraca wrapper

<function __main__.modify_division_function_to_handle_division_by_zero.<locals>.wrapper(a, b)>

In [17]:
modify_division_function_to_handle_division_by_zero(divide)(6, 3)

2.0

In [19]:
modify_division_function_to_handle_division_by_zero(divide)(6, 0)

Funkcji divide jako takiej nigdy nie chcemy wywoływać bo jest podatna na błąd

---

In [32]:
def handle_zero_division(divide_function):
    def wrapper(a, b):
        if b == 0:
            return None
        else:
            return divide_function(a, b)    
    
    return wrapper


@handle_zero_division
def divide(a, b):
    return a / b

In [33]:
divide

<function __main__.handle_zero_division.<locals>.wrapper(a, b)>

In [28]:
divide(6, 3)

2.0

In [29]:
divide(6, 0)

Powyższy zapis jest tak naprawdę wywołaniem funkcji `handle_zero_division` do której przekazujemy `divide`. To zwraca wrapper, do którego następnie przekazujemy argumenty. To co zwróci wrapper jest finalnym wynikiem powyższej linii.

Wydaje się że jest to strzelanie z armaty do wróbla, ale teraz tego walidatora możemy użyć również do innych funkcji, a nie tylko do tej jednej.

In [34]:
def handle_zero_division(func):    # func - zwykle tak piszemy
    def wrapper(*args, **kwargs):  # args, kwargs - jesteśmy w stanie obsłużyć sytuacje gdzie są inne argumenty
        if args[1] == 0:
            return None
        else:
            return func(*args, **kwargs)    
    
    return wrapper


@handle_zero_division
def divide(a, b):
    return a / b


@handle_zero_division
def modulo(a, b):
    return a % b

In [39]:
divide(a=6, b=12)

IndexError: tuple index out of range

In [38]:
modulo(6, 0)

## Przykład 2 – transformacja wartości zwracanej przez funkcję

In [40]:
def return_argument(x):
    return x

In [42]:
return_argument(10)

10

In [43]:
def square(func):
    def wrapper(*args, **kwargs):
        result = func(*args, **kwargs)
        return result ** 2

    return wrapper

In [44]:
@square
def return_argument(x):
    return x

In [45]:
return_argument(4)

16

## Przykład 3 – bardziej abstrakcyjny, obrazuje ideę dekoratora

In [48]:
str.upper

<method 'upper' of 'str' objects>

In [55]:
def my_decorator(_):
    return str.upper



@my_decorator
def my_function(_):
    return 1

Pytanie: co zwroci wywołanie funkcji?

In [56]:
my_function("hello")

'HELLO'

In [52]:
my_function.__name__

'upper'

> **ZADANIA**

## Wiele dekoratorów – przykład 1

In [57]:
import time

In [62]:
time.time()

1725783010.0341914

In [69]:
start = time.perf_counter()

In [70]:
end = time.perf_counter()

In [71]:
import time


def validate_division(func):
    def wrapper(*args, **kwargs):
        if args[1] == 0:
            return None
        else:
            return func(*args, **kwargs)
    
    return wrapper


def measure_time(func):
    def wrapper(*args, **kwargs):
        start = time.perf_counter()
        result = func(*args, **kwargs)
        stop = time.perf_counter()
        print("Czas: ", stop - start)
        return result

    return wrapper


def pure_divide(a, b):
    return a / b


@measure_time
@validate_division
def divide(a, b):
    return a / b

In [77]:
divide(10, 0)

Czas:  5.166002665646374e-06


In [79]:
measure_time(validate_division(pure_divide))(10, 0)

Czas:  1.5830009942874312e-06


---

In [80]:
@validate_division
@measure_time
def divide(a, b):
    return a / b

In [81]:
divide(10, 0)

In [82]:
validate_division(measure_time(pure_divide))(10, 0)

Nie ma printowania czasu, ponieważ tutaj najpierw wywołujemy funkcję walidującą, a ona po sprawdzeniu czy b==0 od razu zwraca None zamiast wywoływać kolejne funkcje.

Kiedy b!=0, czas będzie printowany.

In [83]:
divide(10, 1)

Czas:  1.648004399612546e-06


10.0

## Wiele dekoratorów – przykład 2

In [86]:
def add_exclamation(func):
    def wrapper(*args, **kwargs):
        result = func(*args, **kwargs)
        result += "!"
        return result
    return wrapper


def add_quotation(func):
    def wrapper(*args, **kwargs):
        result = func(*args, **kwargs)
        result += "?"
        return result
    return wrapper


@add_exclamation
@add_quotation
def make_upper(text):
    return text.upper()

In [88]:
make_upper("hello")

'HELLO?!'

In [90]:
add_exclamation(add_quotation(str.upper))("hello")

'HELLO?!'

Wołamy add_exclamation, które woła add_quotation, które woła make_upper

> **ZADANIA**

## Parametryzacja dekoratora

In [94]:
def round_result(func):
    def wrapper(*args, **kwargs):
        result = func(*args, **kwargs)
        rounded_result = round(result)
        return rounded_result

    return wrapper



@round_result
def calculate_average(a, b):
    return (a + b) / 2


calculate_average(1.242342, 2.23)

2

---

In [102]:
def actual_rounding_decorator(param):
    def round_result(func):
        return lambda *args, **kwargs: round(func(*args, **kwargs), param)
    return round_result


@log
@actual_rounding_decorator(3)
def calculate_average(a, b):
    return (a + b) / 2


calculate_average(1.242342, 2.23)

NameError: name 'log' is not defined